In [20]:
import sqlite3
import pandas as pd
con = sqlite3.connect("eu-football-database.sqlite")

In [21]:
query = """
SELECT SUM(weight) AS total_weight
FROM Player;
"""
total_weight = pd.read_sql(query, con)
total_weight

,total_weight
0,1862286


In [22]:
query = """
SELECT AVG(height) AS avg_height
FROM Player;
"""
avg_height = pd.read_sql(query, con)
avg_height

,avg_height
0,181.867445


In [23]:
query = """
SELECT COUNT(*) AS total_teams
FROM Team
"""
total_teams = pd.read_sql(query, con)
total_teams

,total_teams
0,299


In [24]:
query = """
SELECT MIN(date) AS earliest_match
FROM  Match
"""
earliest_match = pd.read_sql(query, con)
earliest_match

,earliest_match
0,2008-07-18 00:00:00


In [25]:
query = """
SELECT MIN(date) AS latest_match
FROM  Match
"""
latest_match = pd.read_sql(query, con)
latest_match

,latest_match
0,2008-07-18 00:00:00


In [26]:
query = """
SELECT preferred_foot, AVG(overall_rating) mean_rating
FROM Player_Attributes
GROUP BY preferred_foot
"""
mean_rating_by_foot_pref = pd.read_sql(query, con)
mean_rating_by_foot_pref

,preferred_foot,mean_rating
0,None,NaN
1,left,68.626182
2,right,68.591558


In [27]:
query = """
SELECT preferred_foot, MAX(overall_rating) max_rating
FROM Player_Attributes
GROUP BY preferred_foot
"""
max_rating_by_foot_pref = pd.read_sql(query, con)
max_rating_by_foot_pref

,preferred_foot,max_rating
0,None,NaN
1,left,94.0
2,right,93.0


In [28]:
query = """
SELECT buildUpPlaySpeedClass, Count(*) AS total_teams
FROM Team_Attributes
GROUP BY buildUpPlaySpeedClass
HAVING defenceAggression >= 40
"""
bupsc_teams_over_40_da = pd.read_sql(query, con)
bupsc_teams_over_40_da

,buildUpPlaySpeedClass,total_teams
0,Balanced,1184
1,Fast,172


In [29]:
query = """
SELECT  p.player_name, 
        AVG(a.potential - a.overall_rating) 
        AS potential_diff,
        p.birthday
FROM Player p
JOIN Player_Attributes a 
ON p.player_api_id = a.player_api_id
WHERE a.date = (
    SELECT date 
    FROM Player Attributes 
    WHERE overall_rating > potential
    )
GROUP BY p.player_name 
ORDER BY potential_diff ASC
LIMIT 10   
"""
avg_potential_diff = pd.read_sql(query, con)
avg_potential_diff

,player_name,potential_diff,birthday
0,Benjamin Huebner,-16.000000,1989-07-04 00:00:00
1,Maikel van der Werff,-13.000000,1989-04-22 00:00:00
2,Pascal Chimbonda,-13.000000,1979-02-21 00:00:00
3,Matt Moussilou,-12.333333,1982-06-01 00:00:00
4,Rafik Saifi,-12.000000,1975-02-07 00:00:00
5,Jason Koumas,-11.600000,1979-09-25 00:00:00
6,Andrea Parola,-11.000000,1979-04-22 00:00:00
7,Derek Riordan,-11.000000,1983-01-16 00:00:00
8,Kanga Akale,-10.500000,1981-03-07 00:00:00
9,Aruna Dindane,-10.333333,1980-11-26 00:00:00


In [30]:
query = """
SELECT id, date, overall_rating, AVG(overall_rating) 
        OVER(
        PARTITION BY date
        ORDER BY overall_rating) AS avg_overall_rating
FROM Player_Attributes;
"""
avg_potential_diff = pd.read_sql(query, con)
avg_potential_diff

,id,date,overall_rating,avg_overall_rating
0,484,2007-02-22 00:00:00,NaN,NaN
1,984,2007-02-22 00:00:00,NaN,NaN
2,1753,2007-02-22 00:00:00,NaN,NaN
3,2090,2007-02-22 00:00:00,NaN,NaN
4,2483,2007-02-22 00:00:00,NaN,NaN
...,...,...,...,...
183973,153509,2016-07-07 00:00:00,68.0,65.600000
183974,1330,2016-07-07 00:00:00,72.0,67.428571
183975,19676,2016-07-07 00:00:00,72.0,67.428571
183976,57230,2016-07-07 00:00:00,73.0,68.125000


In [31]:
query = """
SELECT p.player_name, a.overall_rating
 overall_rating,
 RANK () OVER ( 
     ORDER BY overall_rating DESC
 ) rating
FROM Player_Attributes a
JOIN player p ON a.player_api_id = p.player_api_id
GROUP BY p.player_name
LIMIT 10
"""
rank_overall_rating = pd.read_sql(query, con)
rank_overall_rating

,player_name,overall_rating,rating
0,Lionel Messi,94,1
1,Cristiano Ronaldo,93,2
2,Neymar,90,3
3,Manuel Neuer,90,3
4,Luis Suarez,90,3
5,Zlatan Ibrahimovic,89,6
6,Arjen Robben,89,6
7,Thiago Silva,88,8
8,Sergio Aguero,88,8
9,Robert Lewandowski,88,8


In [32]:
query = """
SELECT  birthday,
     weight,
     AVG(weight) OVER (
     ORDER BY birthday 
     ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
     ) average_weight
FROM Player;
"""
weight_vs_bday_moving_average = pd.read_sql(query, con)
weight_vs_bday_moving_average

,birthday,weight,average_weight
0,1967-01-23 00:00:00,161,174.000000
1,1968-06-26 00:00:00,187,177.000000
2,1969-01-15 00:00:00,183,181.333333
3,1969-03-13 00:00:00,174,180.000000
4,1969-04-01 00:00:00,183,179.333333
...,...,...,...
11055,1998-12-16 00:00:00,157,160.333333
11056,1998-12-20 00:00:00,148,158.333333
11057,1999-01-23 00:00:00,170,172.000000
11058,1999-02-25 00:00:00,198,169.666667


In [33]:
query = """
SELECT  birthday,
     weight,
     AVG(height) OVER (
     ORDER BY birthday 
     ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
     ) average_weight
FROM Player;
"""
height_vs_bday_moving_average = pd.read_sql(query, con)
height_vs_bday_moving_average

,birthday,weight,average_weight
0,1967-01-23 00:00:00,161,185.420000
1,1968-06-26 00:00:00,187,186.266667
2,1969-01-15 00:00:00,183,184.573333
3,1969-03-13 00:00:00,174,182.033333
4,1969-04-01 00:00:00,183,182.033333
...,...,...,...
11055,1998-12-16 00:00:00,157,188.806667
11056,1998-12-20 00:00:00,148,187.113333
11057,1999-01-23 00:00:00,170,188.806667
11058,1999-02-25 00:00:00,198,190.500000


In [34]:
query="""
SELECT  m.id, 
     c.name AS country_name, 
     l.name AS league_name, 
     season, 
     stage, 
     date,
     hmt.team_long_name AS home_team,
     awt.team_long_name AS away_team,
     home_team_goal, 
SUM(home_team_goal) OVER (ORDER BY m.date) 
     AS cumulative_home_goals,
 away_team_goal,
SUM(away_team_goal) OVER (ORDER BY m.date) 
     AS cumulative_away_goals
FROM Match m
JOIN Country c ON c.id = m.country_id
JOIN League l ON l.id = m.league_id
LEFT JOIN Team AS hmt 
     ON hmt.team_api_id = m.home_team_api_id
LEFT JOIN Team AS awt 
     ON awt.team_api_id = m.away_team_api_id
"""
home_vs_away_cumulative_goals = pd.read_sql(query, con)
home_vs_away_cumulative_goals

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,cumulative_home_goals,away_team_goal,cumulative_away_goals
0,24559,Switzerland,Switzerland Super League,2008/2009,1,2008-07-18 00:00:00,BSC Young Boys,FC Basel,1,1,2,2
1,24560,Switzerland,Switzerland Super League,2008/2009,1,2008-07-19 00:00:00,FC Aarau,FC Sion,3,4,1,3
2,24561,Switzerland,Switzerland Super League,2008/2009,1,2008-07-20 00:00:00,FC Luzern,FC Vaduz,1,6,2,7
3,24562,Switzerland,Switzerland Super League,2008/2009,1,2008-07-20 00:00:00,Neuchâtel Xamax,FC Zürich,1,6,2,7
4,24613,Switzerland,Switzerland Super League,2008/2009,2,2008-07-23 00:00:00,FC Basel,Grasshopper Club Zürich,1,9,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25945,Switzerland,Switzerland Super League,2015/2016,36,2016-05-25 00:00:00,FC Basel,Grasshopper Club Zürich,0,40127,1,30160
25975,25946,Switzerland,Switzerland Super League,2015/2016,36,2016-05-25 00:00:00,Lugano,FC St. Gallen,3,40127,0,30160
25976,25947,Switzerland,Switzerland Super League,2015/2016,36,2016-05-25 00:00:00,FC Luzern,FC Sion,2,40127,2,30160
25977,25948,Switzerland,Switzerland Super League,2015/2016,36,2016-05-25 00:00:00,FC Thun,BSC Young Boys,0,40127,3,30160


In [35]:
query = """
SELECT 
    player_name,
    weight / ((height / 100)* (height / 100)) AS BMI,
    NTILE(100) OVER(ORDER BY 
       (weight / ((height / 100)* (height / 100)))) 
       BMI_rank
FROM 
    Player;
"""
bmi_percentile = pd.read_sql(query, con)
bmi_percentile

,player_name,BMI,BMI_rank
0,John Stewart,38.778383,1
1,Mariusz Rybicki,39.393063,1
2,Harry Kane,40.476706,1
3,Christopher Teichmann,40.489877,1
4,Adam Gyurcso,40.587266,1
...,...,...,...
11055,Oswal Alvarez,63.810252,100
11056,George Elokobi,65.185609,100
11057,Jason Brown,65.185609,100
11058,Tim Wiese,65.209618,100


In [36]:
query = """
SELECT m.id, 
    c.name AS country_name, 
    l.name AS league_name, 
    season, 
    stage, 
    date,
    hmt.team_long_name AS home_team,
    awt.team_long_name AS away_team,
    home_team_goal, 
    away_team_goal                                        
FROM Match m
JOIN Country c 
        ON c.id = m.country_id
JOIN League l 
        ON l.id = m.league_id
LEFT JOIN Team AS hmt 
        ON hmt.team_api_id = m.home_team_api_id
LEFT JOIN Team AS awt
        ON awt.team_api_id = m.away_team_api_id
"""
match_join = pd.read_sql(query)
match_join

TypeError: read_sql() missing 1 required positional argument: 'con'

In [37]:
query = """
SELECT CASE
    WHEN ROUND(height)<165 THEN 165
    WHEN ROUND(height)>195 THEN 195
    ELSE ROUND(height)
    END AS calc_height, 
    COUNT(height) AS distribution, 
    (AVG(group_pa.avg_overall_rating)) 
        AS avg_overall_rating,
    (AVG(group_pa.avg_potential)) 
        AS avg_potential,
    AVG(weight) 
        AS avg_weight 
FROM Player p
LEFT JOIN (SELECT a.player_api_id, 
    AVG(a.overall_rating) 
        AS avg_overall_rating,
    AVG(a.potential) 
        AS avg_potential  
    FROM Player_Attributes a
    GROUP BY a.player_api_id) 
        AS group_pa 
        ON p.player_api_id = group_pa.player_api_id
GROUP BY calc_height
ORDER BY calc_height
"""
case_rating_vs_height = pd.read_sql(query, con)
case_rating_vs_height

,calc_height,distribution,avg_overall_rating,avg_potential,avg_weight
0,165.0,74,67.365543,73.327754,139.459459
1,168.0,118,67.500518,73.124182,144.127119
2,170.0,403,67.726903,73.379056,147.799007
3,173.0,530,66.980272,72.848746,152.824528
4,175.0,1188,66.805204,72.258774,156.111953
5,178.0,1489,66.367212,71.943339,160.665547
6,180.0,1388,66.419053,71.846394,165.261527
7,183.0,1954,66.634380,71.754555,170.167861
8,185.0,1278,66.928964,71.833475,174.636933
9,188.0,1305,67.094253,72.151949,179.278161
